# Retrive ES data for Bruppleinitial and Burpple
    save the object with pickle

    >'burpple__review__unique__***today***.p',   
    

## 1. Conenct with ES

In [1]:
# set up ElasticSearch object and the URL to access it
def setup_es(isServer):
    # python version
    import sys
    print "system_info: %s"%sys.version
    # current working directory
    import os
    print "path_info: %s"%os.getcwd()    
    ## Local on PC/laptop or on VM (10.0.106.122:2)   
    from elasticsearch import Elasticsearch
    port = "9200"
    host = "localhost"
    if isServer:
        host = "10.0.109.54"    
    url = "http://" + host + ":" + port    
    print "es_info: %s"%url    
    es = Elasticsearch([{'host': host, 'port': port}])
    return es, url

es, url = setup_es(True)

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue
es_info: http://10.0.109.54:9200


In [2]:
# make sure ES is up and running
import requests
def initialise_es(i):
    res = requests.get(url)
    if i:
        print(res.content)
initialise_es(1)

{
  "name" : "star7",
  "cluster_name" : "star",
  "cluster_uuid" : "VDcV69ThThyb00sd6PgeMA",
  "version" : {
    "number" : "2.4.5",
    "build_hash" : "c849dd13904f53e63e88efc33b2ceeda0b6a1276",
    "build_timestamp" : "2017-04-24T16:18:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.4"
  },
  "tagline" : "You Know, for Search"
}



In [3]:
# define dict for b and bi
burpple = {"_index":"burpple"}
burppleinitial = {"_index":"burppleinitial"}
burpples = [burpple, burppleinitial]
for website in burpples:
    website.update({"vendor":"restaurant",
                    "review":"review",  
                    "user":"user",                                   
                    "_id":"_id",
                    "vendor_id":'_source.id',
                    "vendor_url":'_source.url',
                    "vendor_name":"_source.name", 
                    "address":"_source.address", 
                    "cuisine_tags":"_source.tags", 
                    "phone":"_source.phone",                
                    "cycleStart":'_source.crawlTimeStamp'})    

In [4]:
burpple.update({"reviewFeedTime":'_source.feedDatetime',})
burppleinitial.update({"reviewFeedTime":'_source.datetime',})

## 2. Actual data retrival

In [5]:
def retrive_data (website, doc_type):
    # activate ES
    initialise_es(0)
    # ES search pattern
    _body = {"query": {"match_all": {}}}
    _index = website['_index']
    _doc_type = website[doc_type]

    # With the help of a generator, get all records
    from elasticsearch import helpers
    scanResp = helpers.scan(es, _body, scroll= "2m", 
                            index= _index, 
                            doc_type= _doc_type, 
                            timeout="2m")
    recs = []
    for rec in scanResp:
        recs.append(rec)
    print len(recs)

    # Convert unicode to string (ascii, ignore unicode such as '\xae')
    def convert_unicode(data):
        if isinstance(data, basestring):
            return (data.encode("ascii","ignore"))
        elif isinstance(data, collections.Mapping):
            return dict(map(convert_unicode, data.iteritems()))
        elif isinstance(data, collections.Iterable):
            return type(data)(map(convert_unicode, data))
        else:
            return data

    # json file to dataframe 
    import collections
    import pandas as pd
    pd.options.mode.chained_assignment = None # default is warn
    from pandas.io.json import json_normalize
    lst_rec = []
    for line in recs:
        line = convert_unicode(line)
        lst_rec.append(json_normalize(line))    
    df = pd.concat(lst_rec) 
    # Print shape and all attributes
    print ("ES location: %s, %s"%(_index,website[doc_type]))
    print("Dimention: %d , %d"%df.shape)
    print("Column names::%s"%", ".join(df.columns.tolist()))
    print ("\n")
    
    return df

In [6]:
%%time
initialise_es(1)
print ""

website = burpple
website.update({"all_rec_review" : retrive_data (website, "review")})

{
  "name" : "star7",
  "cluster_name" : "star",
  "cluster_uuid" : "VDcV69ThThyb00sd6PgeMA",
  "version" : {
    "number" : "2.4.5",
    "build_hash" : "c849dd13904f53e63e88efc33b2ceeda0b6a1276",
    "build_timestamp" : "2017-04-24T16:18:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.4"
  },
  "tagline" : "You Know, for Search"
}


25709
ES location: burpple, review
Dimention: 25709 , 43
Column names::_id, _index, _score, _source.body, _source.crawlTimeStamp, _source.feedDatetime, _source.foodImgUrl, _source.id, _source.name, _source.postDatetime, _source.restaurant, _source.restaurant.address, _source.restaurant.areaUrls, _source.restaurant.avgPrice, _source.restaurant.crawlTimeStamp, _source.restaurant.id, _source.restaurant.name, _source.restaurant.openingHours, _source.restaurant.phone, _source.restaurant.tags, _source.restaurant.url, _source.restaurant.website, _source.restaurantUrl, _source.title, _source.url, _source.user, _source.user.awards, _source.user.badg

In [7]:
%%time
website = burppleinitial
website.update({"all_rec_review" : retrive_data (website, "review")})

297480
ES location: burppleinitial, review
Dimention: 297480 , 44
Column names::_id, _index, _score, _source.areaName, _source.body, _source.crawlTimeStamp, _source.datetime, _source.foodImgUrl, _source.id, _source.name, _source.offset, _source.restaurant, _source.restaurant.address, _source.restaurant.areaUrls, _source.restaurant.avgPrice, _source.restaurant.crawlTimeStamp, _source.restaurant.id, _source.restaurant.name, _source.restaurant.openingHours, _source.restaurant.phone, _source.restaurant.tags, _source.restaurant.url, _source.restaurant.website, _source.restaurantUrl, _source.title, _source.url, _source.user, _source.user.awards, _source.user.badges, _source.user.bio, _source.user.country, _source.user.crawlTimeStamp, _source.user.name, _source.user.numOfFollowers, _source.user.numOfFollowing, _source.user.numOfReviews, _source.user.numOfWishlists, _source.user.url, _source.user.userImgUrl, _source.user.username, _source.user.website, _source.userImgUrl, _source.username, _ty

## 3. Duplicate records removal  

In [8]:
def remove_bu_dup_records(burpples, non_sg_vendors):
    # not included: '_source.url', # https://www.burpple.com/f/ + "_id"
    cols = ['_id', # review identifier, something like "liKrL-pE"
            '_type',
            '_index', # burpple / burpple initial
            '_source.title', # title of review, with some special characters
            '_source.body', # text 
            '_source.crawlTimeStamp', 
            '_source.foodImgUrl',             
            '_source.username', # user identifier
            '_source.restaurant.id',
            '_source.restaurant.name'] #vendor identifier
    # merge feed time
    import pandas as pd
    pd.options.mode.chained_assignment = None # default is warn
    df = pd.concat([site["all_rec_review"]
                    [cols+ [site["reviewFeedTime"]]] for site in burpples])
    df["feedTime"] = df[[site["reviewFeedTime"] for site in burpples]].fillna('').sum(axis=1)
    df = df[cols +["feedTime"]]
    # sort, only leaving the latest crawled first
    df = df.sort_values(by=['_source.crawlTimeStamp',"_id"], ascending=[False,True])
    df = df.groupby("_id").first()
    df.reset_index(inplace=True)
    # remove reviews of non-sg vendors
    df = df[~df["_source.restaurant.id"].isin(non_sg_vendors)]
    
    print ("Got %d unique records of review" % len(df))
    print ("Got %d unique records of images" % 
           df[df['_source.foodImgUrl']!=""]['_source.foodImgUrl'].nunique())
    print ("Got %d unique users" % df['_source.username'].nunique())
    print ("Got %d unique vendors" % df['_source.restaurant.id'].nunique())
    print ("Review time from %s to %s" % (df['feedTime'].min(),df['feedTime'].max()))
    return df
    

In [9]:
%%time
non_sg_vendors = ['114803', '119954', '136868', '139058', '149618', '155202', '156512', '16431',
                  '165933', '166041', '174029', '28318', '43464', '51131', '59732', '63212']
df = remove_bu_dup_records(burpples, non_sg_vendors)

Got 323104 unique records of review
Got 323051 unique records of images
Got 11180 unique users
Got 14547 unique vendors
Review time from 2011-11-08 00:00:00.000000 to 2017-07-20 23:04:45.837391
CPU times: user 8.96 s, sys: 764 ms, total: 9.73 s
Wall time: 9.73 s


In [10]:
df.head(3)

,_id,_type,_index,_source.title,_source.body,_source.crawlTimeStamp,_source.foodImgUrl,_source.username,_source.restaurant.id,_source.restaurant.name,feedTime
0,--0EG2cp,review,burppleinitial,Sicilian Lemon Creme andCaramelized Red Mixed ...,,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/376d87a944deb0499...,@ieatkows,34166,OSO Ristorante,2014-12-22 00:00:00.000000
1,--0ck5YP,review,burppleinitial,Just look at their soy sauce chicken.,The tender and juicy thigh meat with their fra...,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/2fbab5e890cf97f31...,@jiaknonstop,164762,Fatty Ox Hong Kong Kitchen (Chinatown Complex ...,2014-12-15 00:00:00.000000
2,--0gMGRO,review,burppleinitial,"Holy, smokin' Burnt Corn Tacos - it's their WE...",(Read: time for MORE alcohol.) #AntiDoteBar #...,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/16ff778f20e5f8614...,@Jazpster,539,Asian Market Caf (Fairmont Singapore),2016-11-18 00:00:00.000000


## 4. Save Files

In [11]:
def save_files(df, doc_type):
    # data description as file name
    import datetime
    today = datetime.date.today()
    today = today.strftime('%Y%m%d')
    desc = "__".join(doc_type + [today])  
    file_name = desc + ".p"
    print file_name
    # save the fil`e as pickle file
    import pickle
    with open(file_name, 'wb') as pfile:
        pickle.dump(df, pfile)

In [12]:
%%time
doc_type = ["burpple", "review", "unique"]
save_files(df, doc_type)

burpple__review__unique__20180202.p
CPU times: user 18.7 s, sys: 1.16 s, total: 19.9 s
Wall time: 19.9 s


In [19]:
# retrive for testing
def retrive_file(file_name):
    import pickle
    with open(file_name, 'rb') as pfile:
        retrived = pickle.load(pfile)
    return retrived

## Further cleaning

In [20]:
file_name = "burpple__review__unique__20180202.p"    
df_retrived = retrive_file(file_name)

## 3. Obtain review objects with clean text

In [21]:
import pandas as pd
pd.options.mode.chained_assignment = None # default is warn
# new col: loc
df["loc"] = df["_index"] + "/" + df["_type"] + "/" + df["_id"] 
cols = ['loc','_source.title','_source.body','_source.restaurant.name']
df = df[cols]
rename_cols = [col.replace("_source.", "") for col in cols]
df.columns = rename_cols
# new col: full_review
df["full_review"] = df['title'] + " " + df['body']

In [22]:
""" 
twitter style text-cleaning, for url, mention, hashtag, 
also: dollar, score
not: apostrophe conversion, stop words, emoticons, slang, word standardization
"""
def clean_review(s):
    import re   
    # 1. etract urls (may contain $, #, @), remove url part from s
    p = r'http[s]?://(?:[a-z]|[0-9]|[$-_@#.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
    urls = tuple(re.findall(p, s))
    for rep in urls:
        s = s.replace(rep,"")
    # 2. normal cleaning: replace w/ and &
    s = " " + s +" "
    s = s.replace("w/", " with ")
    s = s.replace("&", " and ") 
    # 3. textract hashtags 
    hashtags = tuple(re.findall(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", s))
    # 4. extract mentions
    mentions = tuple(re.findall(r'(?:@[\w_]+)', s))   
    # 5. extract dolalr amounts
    dollars = tuple([x[0] for x in re.findall(r'(\$\d+([,\.]\d+)?(\+\+)?(\+)?k?)', s)])
    # 6. extract scores
    scores = tuple([x[0] for x in re.findall(r'(\d+([\.]\d+)(\/)\d+([\.]\d+)?)', s)])
    # 7. remove above parts from review, strip and remove multiple space
    words = sorted(list(dollars + scores + hashtags + mentions), key=len, reverse=True)
    for rep in words:
        s = s.replace(rep,"")
    s = re.sub(' +',' ', s.strip()) 
    lst = [s, hashtags, mentions, dollars, scores, urls]
    return (tuple(lst))

In [23]:
%%time
df["review_text"], df["hashtags"], df["mentions"], df["dollars"], df["scores"], df["urls"] = \
zip(*df["full_review"].apply(clean_review))
cols = df.columns.tolist()
cols.remove("full_review")
df = df[cols]

CPU times: user 15.7 s, sys: 460 ms, total: 16.2 s
Wall time: 16.1 s


In [24]:
print df.shape
print df.columns.tolist()

(323104, 10)
['loc', 'title', 'body', 'restaurant.name', 'review_text', 'hashtags', 'mentions', 'dollars', 'scores', 'urls']


In [25]:
df.head(3)

,loc,title,body,restaurant.name,review_text,hashtags,mentions,dollars,scores,urls
0,burppleinitial/review/--0EG2cp,Sicilian Lemon Creme andCaramelized Red Mixed ...,,OSO Ristorante,Sicilian Lemon Creme andCaramelized Red Mixed ...,"(#igsg, #igers, #igfood, #yum, #yummy, #instaf...",(),(),(),()
1,burppleinitial/review/--0ck5YP,Just look at their soy sauce chicken.,The tender and juicy thigh meat with their fra...,Fatty Ox Hong Kong Kitchen (Chinatown Complex ...,Just look at their soy sauce chicken. The tend...,(),(),(),(),()
2,burppleinitial/review/--0gMGRO,"Holy, smokin' Burnt Corn Tacos - it's their WE...",(Read: time for MORE alcohol.) #AntiDoteBar #...,Asian Market Caf (Fairmont Singapore),"Holy, smokin' Burnt Corn Tacos - it's their WE...","(#AntiDoteBar, #FairmontSingapore)",(),(),(),()


In [26]:
def save_files(df, doc_type):
    # data description as file name
    import datetime
    today = datetime.date.today()
    today = today.strftime('%Y%m%d')
    desc = "__".join(doc_type+[today])  
    file_name = desc + ".p"
    print file_name
    # save the fil`e as pickle file
    import pickle
    with open(file_name, 'wb') as pfile:
        pickle.dump(df, pfile)

In [27]:
%%time
doc_type = ["burpple", "review", "clean"]
save_files(df, doc_type)

burpple__review__clean__20180205.p
CPU times: user 17.5 s, sys: 464 ms, total: 18 s
Wall time: 18 s


In [28]:
%%time
file_name = "burpple__review__clean__20180205.p"    
df_retrived = retrive_file(file_name)

CPU times: user 17.6 s, sys: 300 ms, total: 17.9 s
Wall time: 17.9 s
